![Python Logo](../../assets/images/python-logo-master-v3-TM.png)

# <center> Exploratory Data Analysis for AskUbuntu Question Data </center>
### <center> Shagun Sodhani </center>

In [1]:
# put in the name of the Stack Exchange Site which you want to analyse.
data_source = "AskUbuntu"
data_type = "Question"
base_url = "http://askubuntu.com/questions/tagged/"

## Let us begin

### We will import everything in one cell for the sake for reference.

In [2]:
import pyspark.sql.functions as func
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.mllib.fpm import FPGrowth
import pandas as pd
from time import time
import operator
%matplotlib inline
from __future__ import print_function

### Now let us load the question data as a dataframe. Since we will be using this data many times, we will cache it in memory.

In [5]:
start_time = time()
print ("data_source =", data_source)
print ("data_type =", data_type)
question_data_path = "../../data/"+data_source+"/"+data_type
print ("question_data_path =",question_data_path)
question_df = sqlContext.read.load(question_data_path).cache()
print ("Time taken =", time() - start_time, "seconds.")

data_source = AskUbuntu
data_type = Question
question_data_path = ../../data/AskUbuntu/Question
Time taken = 0.212697029114 seconds.


In [6]:
print("Schema for question dataframe")
question_df.printSchema()

Schema for question dataframe
root
 |-- Id: long (nullable = true)
 |-- AcceptedAnswerId: long (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- Score: long (nullable = true)
 |-- ViewCount: long (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: long (nullable = true)
 |-- LastEditorUserId: long (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- LastEditDate: string (nullable = true)
 |-- LastActivityDate: string (nullable = true)
 |-- CommunityOwnedDate: string (nullable = true)
 |-- ClosedDate: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- AnswerCount: long (nullable = true)
 |-- CommentCount: long (nullable = true)
 |-- FavoriteCount: long (nullable = true)



### We want to play with the tag data.

In [7]:
question_df.select('Tags').show(10, truncate = False)

+----------------------------------------+
|Tags                                    |
+----------------------------------------+
|<updates><unattended-upgrades>          |
|<wallpaper><artwork><photography>       |
|<package-management><uninstall>         |
|<command-line><package-management>      |
|<ntp>                                   |
|<applet><panel><networking>             |
|<metacity><window-buttons>              |
|<wine><browser><ie>                     |
|<thunderbird><google-calendar><calendar>|
|<firewall>                              |
+----------------------------------------+
only showing top 10 rows



### We need to clean up this data. Specifically, we want to remove the "<>" signs. We will use the [regex_replace](https://spark.apache.org/docs/1.5.2/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_replace) function.

In [8]:
question_df.select(func.regexp_replace\
                   (func.regexp_replace('Tags', '><', ','), '<|>', '').alias('Tags')).show(10, truncate = False)

+------------------------------------+
|Tags                                |
+------------------------------------+
|updates,unattended-upgrades         |
|wallpaper,artwork,photography       |
|package-management,uninstall        |
|command-line,package-management     |
|ntp                                 |
|applet,panel,networking             |
|metacity,window-buttons             |
|wine,browser,ie                     |
|thunderbird,google-calendar,calendar|
|firewall                            |
+------------------------------------+
only showing top 10 rows



### But we do not want just tags. We want other metrics like Score, ViewCount etc as well.


In [9]:
column_list = ["Score", "ViewCount", "AnswerCount", "CommentCount", "FavoriteCount"]
tags_df = question_df.select("Score", "ViewCount", "AnswerCount", "CommentCount", "FavoriteCount", func.regexp_replace\
                   (func.regexp_replace('Tags', '><', ','), '<|>', '').alias('Tags'))
tags_df.show(10, truncate = False)

+-----+---------+-----------+------------+-------------+------------------------------------+
|Score|ViewCount|AnswerCount|CommentCount|FavoriteCount|Tags                                |
+-----+---------+-----------+------------+-------------+------------------------------------+
|88   |44595    |5          |1           |27           |updates,unattended-upgrades         |
|16   |641      |1          |2           |2            |wallpaper,artwork,photography       |
|11   |3571     |3          |1           |2            |package-management,uninstall        |
|147  |26759    |6          |1           |40           |command-line,package-management     |
|17   |37191    |3          |1           |10           |ntp                                 |
|2    |3048     |2          |1           |0            |applet,panel,networking             |
|9    |1186     |3          |0           |2            |metacity,window-buttons             |
|12   |5807     |0          |0           |0            |wine

### Now we want to have multiple rows corresponding to each row of *Tags* such that each row has only one *Tag* and all the metrics belonging to the original row. We will use the [split function](https://spark.apache.org/docs/1.5.2/api/python/pyspark.sql.html#pyspark.sql.functions.split) and [explode function](https://spark.apache.org/docs/1.5.2/api/python/pyspark.sql.html#pyspark.sql.functions.explode).

In [10]:
tag_df = tags_df.select("Score", "ViewCount", "AnswerCount", "CommentCount", "FavoriteCount", func.explode(\
        func.split('Tags', ',')).alias('Tag')).cache()
tag_df.show(10, truncate = False)

+-----+---------+-----------+------------+-------------+-------------------+
|Score|ViewCount|AnswerCount|CommentCount|FavoriteCount|Tag                |
+-----+---------+-----------+------------+-------------+-------------------+
|88   |44595    |5          |1           |27           |updates            |
|88   |44595    |5          |1           |27           |unattended-upgrades|
|16   |641      |1          |2           |2            |wallpaper          |
|16   |641      |1          |2           |2            |artwork            |
|16   |641      |1          |2           |2            |photography        |
|11   |3571     |3          |1           |2            |package-management |
|11   |3571     |3          |1           |2            |uninstall          |
|147  |26759    |6          |1           |40           |command-line       |
|147  |26759    |6          |1           |40           |package-management |
|17   |37191    |3          |1           |10           |ntp                |

### Next we [groupBy](https://spark.apache.org/docs/1.5.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) this data using *Tag* as the key and find out the aggregations corresponding to each of the metrics.

In [12]:
agg_list = ["sum"]*(len(column_list))
aggregation_map = (dict(zip(column_list, agg_list)))
groupded_tag_df = tag_df.groupBy('Tag').agg(aggregation_map)
groupded_tag_df.describe().toPandas()

,summary,sum(AnswerCount),sum(ViewCount),sum(Score),sum(CommentCount),sum(FavoriteCount)
0,count,3012,3012,3012,3012,3012
1,mean,262.5996015936255,626007.1580345286,444.0504648074369,325.5401726427623,140.08333333333334
2,stddev,1074.8681018802083,2700600.292591697,1762.805850455362,1480.240786885741,567.6228558096535
3,min,0,10,-5,0,0
4,max,23289,72136113,46803,38587,14690


### Disclaimer - Pandas is used only for visualising the processed data in tabular form as Jupyter notebooks visualise Pandas dataframes very nicely. All the processing is being done by Spark.

### Now we want to rank tags by order of their score so that we know which tags result in high scoring questions.

In [13]:
groupded_tag_df.select('Tag', 'sum(Score)').orderBy('sum(Score)').toPandas()

,Tag,sum(Score)
0,festival,-5
1,cmd,-5
2,kali,-4
3,backtrack,-2
4,m2,-2
5,pdfsam,-1
6,heimdall,-1
7,iperf,-1
8,case-insensitive,-1
9,hexdump,-1


### We can repeat this excercise for any of our metric.

In [14]:
groupded_tag_df.select('Tag', 'sum(ViewCount)').orderBy('sum(ViewCount)').toPandas()

,Tag,sum(ViewCount)
0,texdoc,10
1,corebird,10
2,usb-ethernet,12
3,pdfsam,14
4,exceed-on-demand,17
5,low-latency,17
6,terminal-plus,20
7,biometrics,20
8,ros,20
9,gjiten,20


### Instead of sum, we want to see the average value of these metrics.

In [15]:
agg_list = ["avg"]*(len(column_list))
aggregation_map = (dict(zip(column_list, agg_list)))
groupded_tag_df = tag_df.groupBy('Tag').agg(aggregation_map)
groupded_tag_df.describe().toPandas()

,summary,avg(AnswerCount),avg(ViewCount),avg(Score),avg(CommentCount),avg(FavoriteCount)
0,count,3012,3012,3012,3012,3012
1,mean,1.261576813100134,2658.697828055242,2.410384949150589,1.425766698023223,0.714533869149954
2,stddev,0.5133550607662078,3009.932010924707,2.4140642082317814,0.9902837728225818,0.8302195305978429
3,min,0.0,10.0,-2.0,0.0,0.0
4,max,8.0,32894.43939393939,34.3,16.0,14.0


In [16]:
groupded_tag_df.select('Tag', 'avg(Score)').orderBy('avg(Score)').toPandas()

,Tag,avg(Score)
0,m2,-2.000000
1,pdfsam,-1.000000
2,hexdump,-1.000000
3,cmd,-1.000000
4,festival,-0.833333
5,heimdall,-0.500000
6,iperf,-0.500000
7,coreboot,-0.500000
8,case-insensitive,-0.333333
9,backtrack,-0.086957


In [17]:
groupded_tag_df.select('Tag', 'avg(AnswerCount)').orderBy('avg(AnswerCount)').toPandas()

,Tag,avg(AnswerCount)
0,qgifer,0.000000
1,terminal-plus,0.000000
2,font-converter,0.000000
3,usb-dac,0.000000
4,crimson,0.000000
5,hive,0.000000
6,symbian,0.000000
7,mugshot,0.000000
8,openstack-trove,0.000000
9,distributed-system,0.000000


## <center> [Frequent Patterns](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.fpm.FPGrowth) </center>

### Frequent Pattern - Set of tags which frequently occur together
### Support - Measure of how frequently the tag appears in the data

### First we need to process our data to make it look like transcational data. We will map each row to array of tags. Each array of tags would define one transaction.

In [18]:
tags_transaction = tags_df.select(func.split('Tags', ',').alias('Tags')).map(lambda line: line[0]).cache()
for transaction in tags_transaction.take(5):
    print (transaction)
print ("Number of questions =",tags_transaction.count())

[u'updates', u'unattended-upgrades']
[u'wallpaper', u'artwork', u'photography']
[u'package-management', u'uninstall']
[u'command-line', u'package-management']
[u'ntp']
Number of questions = 230177


In [19]:
start_time = time()
model = FPGrowth.train(tags_transaction, minSupport=0.01, numPartitions=10)
result = sorted(model.freqItemsets().collect(), key = lambda a: -1*a[1])
for fi in result[:10]:
    print(fi)
print ("Time taken =", time() - start_time, "seconds.")

FreqItemset(items=[u'14.04'], freq=20795)
FreqItemset(items=[u'12.04'], freq=17532)
FreqItemset(items=[u'boot'], freq=11307)
FreqItemset(items=[u'networking'], freq=10355)
FreqItemset(items=[u'command-line'], freq=10342)
FreqItemset(items=[u'dual-boot'], freq=9109)
FreqItemset(items=[u'drivers'], freq=8427)
FreqItemset(items=[u'unity'], freq=8351)
FreqItemset(items=[u'wireless'], freq=8234)
FreqItemset(items=[u'server'], freq=7629)
Time taken = 3.52160596848 seconds.
